# Problem Set 6
By: Bethany Bailey
# 9.6

In [4]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import optimize as opt
from scipy import linalg as la

In [2]:
def steepest_descent_quadratic(Q, b, c, x_0, eps=1e-9, maxiters=1e3):
    ''' Implements Steepest Descent Method for quadratic functions.'''
    dist = 1e5
    i = 0
    x_k = x_0
    
    while dist > eps and i < maxiters:
        DfT = Q @ x_k - b
        alpha_k = (DfT.T @ DfT) / (DfT.T @ Q @ DfT)
        x_k1 = x_k - alpha_k * DfT
        dist = la.norm(DfT.T)
        x_k = x_k1
        i += 1
        
    if i < maxiters:
        print(f"Converged in {i} iterations")
        f = 0.5 * x_k.T @ Q @ x_k - b.T @ x_k + c
    else:
        print("Did not converge")
        f = None
        
    return x_k, f

In [3]:
Q = np.array([[3, 1],[1, 3]])
b = np.array([-3, 1])
c = 0
x_0 = np.array([20, 5])
steepest_descent_quadratic(Q, b, c, x_0)

Converged in 17 iterations


(array([-1.25,  0.75]), -2.25)

In [5]:
# Confirm algorithm
def f(x, args):
    Q, b, c = args
    return 0.5 * x.T @ Q @ x - b.T @ x + c
opt.minimize(f, x_0, args=[Q, b, c])

      fun: -2.249999999996836
 hess_inv: array([[ 0.37528592, -0.1262136 ],
       [-0.1262136 ,  0.38014872]])
      jac: array([  4.47034836e-07,   4.23192978e-06])
  message: 'Optimization terminated successfully.'
     nfev: 36
      nit: 7
     njev: 9
   status: 0
  success: True
        x: array([-1.25000037,  0.75000153])

# 9.7

In [6]:
def compute_Df(f, x, Rerr=1e-9):
    '''Compute Df(x)'''
    step = np.sqrt(Rerr)  
    if np.isscalar(x):
        n = 1
    else:
        n = len(x)
    if np.isscalar(f(x)):
        m = 1
    else:
        m = len(f(x))
    
    Df = np.empty((m, n))
    for j in range(n):
        e = np.identity(n)[j] 
        Df[:, j] = (f(x + step * e) - f(x - step * e)) / (2 * step)
    return Df

In [7]:
f = lambda x: np.array([x[0] ** 2 + x[2], x[0] ** 3 - x[1]]).T
x = np.array([1, 2, 3])
compute_Df(f, x)

array([[ 2.,  0.,  1.],
       [ 3., -1.,  0.]])

# 9.8

In [17]:
def secant(x_0, x_1, ϕ, ɛ=1e-9, maxiters=100):
    '''Secant method for line search.'''
    x = np.zeros(maxiters, dtype=float)
    fprime = np.zeros(maxiters, dtype=float)
    
    x[0] = x_0
    fprime[0] = compute_Df(ϕ, x[0])[0]
    x[1] = x_1
    fprime[1] = compute_Df(ϕ, x[1])[0]
    
    k = 2
    dist = 1e3
    while dist > ɛ and k < maxiters:
        x[k] = x[k-1] - fprime[k-1] * ((x[k-1] - x[k-2]) / (fprime[k-1] - fprime[k-2]))
        fprime[k] = compute_Df(ϕ, x[k])[0]
        dist = np.abs((x[k] - x[k-1]) / x[k-1])
        k +=1
        
    return x[k-1]
    
def steepest_descent(f, x_0, ɛ=1e-6, maxiters=1000):    
    '''Steepest descent method for arbitrary functions.'''
    dist = 1e3
    i = 0
    x_k = x_0
    while dist > ɛ and i < maxiters:
        Df = compute_Df(f, x_k, 1e-3)[0]
        ϕ = lambda α: f(x_k - α * Df.T)
        α_k = secant(0.1, 0.8, ϕ)
        x_k1 = x_k - α_k * Df.T
        dist = la.norm(x_k1 - x_k)
        x_k = x_k1
        i += 1
    if i < maxiters:
        print(f"Converged in {i} iterations")
        min_val = f(x_k)
    else:
        print("Did not converge")
        min_val = None
    return x_k, min_val

# 9.9

In [18]:
f = lambda x: 100 * (x[1] - x[0] ** 2) ** 2 + (1 - x[0]) ** 2
x_0 = np.array([-2, 2])

(x, y), f_xy = steepest_descent(f, x_0)

Converged in 94 iterations


In [19]:
print('Minimizer: (', x, y, ')')
print('Minimum:', f_xy)

Minimizer: ( 0.989370216532 0.978800430994 )
Minimum: 0.000113273136867
